# Practice Assignment 3
# Summarizing Data

For this practice assignment some of the code has been written for you ahead of time.  Cells you just need to run are mark 'Run this cell'.  For each question, there are clear instructions in each cell. Follow those instructions and write the code after each block of:

YOUR CODE HERE

Please use the exact variable name if it is specified in the comment.

We’ll run a Python test script against your program to test whether each function implementation is correct.

In [ ]:
%%capture
###########################################################
### EXECUTE THIS CELL BEFORE YOU TO TEST YOUR SOLUTIONS ###
###########################################################
"""
Loading the necessary python libraries
"""
import imp
import pandas as pd
import numpy as np
from nose.tools import assert_equal
from pandas.util.testing import assert_frame_equal, assert_series_equal
sol = imp.load_compiled("sol", "./.sol.py")

In [ ]:
"""
First we need to load the dataset and preprocess the data. The below code comes from some of the work done in Practice Assignment 01.
"""

#Run this cell
restaurant_location_df = pd.read_csv("geoplaces2.csv")
restaurant_accept_df = pd.read_csv("chefmozaccepts.csv")
restaurant_cuisine_df = pd.read_csv("chefmozcuisine.csv")
rating_final_df = pd.read_csv("rating_final.csv")
user_profile_df = pd.read_csv("userprofile.csv")

restaurant_location_df = restaurant_location_df[["placeID", "name", "address", "city", "state", "country", "price", "alcohol"]]
restaurant_location_df = restaurant_location_df.apply(lambda x: x.astype(str).str.lower())
restaurant_location_df = restaurant_location_df.astype({'placeID':'int',"name":"string", "address":"string","city":"string", "state":"string", "country":"string", "price":"string", "alcohol": "string"})
restaurant_location_df = restaurant_location_df.replace('\?', np.nan, regex=True).dropna().reset_index(drop = True)
user_profile_df = user_profile_df.replace('\?', np.nan, regex=True)
user_profile_df = user_profile_df[['userID', 'smoker', 'drink_level', 'marital_status', 'birth_year']]

#Since The values in column 'smoker' are strings, we have to convert them to boolean values
user_profile_df['smoker'] = user_profile_df['smoker'].map({'true': True, 'false': False})
###

In [ ]:
"""
Lets first examine the columns and datatypes for our "restaurant_location_df" dataframe.
"""

#Run this cell
restaurant_location_df.info()

In [ ]:
"""
Now let's display the "restaurant_location_df" dataframe.  As a reminder, we can use display() to display
dataframes as easy-to-read html tables.
"""

#Run this cell
display(restaurant_location_df)

In [ ]:
"""
To quickly get an idea of the rows in all of the dataframes, we'll use .head() in the following 4 cells.
View the first 5 rows for "restaurant_accept_df".
"""

#Run this cell
restaurant_accept_df.head()

In [ ]:
"""
View the first 5 rows for "restaurant_cuisine_df".
"""

#Run this cell
restaurant_cuisine_df.head()

In [ ]:
"""
View the first 5 rows for "user_profile_df".
"""

#Run this cell
user_profile_df.head()

In [ ]:
"""
View the first 5 rows for "rating_final_df".
"""

#Run this cell
rating_final_df.head()

In [ ]:
"""
Part 01

1. In order to have a better view of the payment methods of each restaurant, we are going to merge
"restaurant_location_df" and "restaurant_accept_df".  Use an inner join for this merge, and merge on 'placeID'.
The resulting dataframe should be called "accept_payment_df" and only contain 2 columns 'name' and 'Rpayment'.

2. Next we want to group by the 'name' column, and update the 'Rpayment' column to contain a comma seperated
string, e.g. 'cash, VISA, MasterCard-Eurocard'.
Hint: Using the agg() function here will help with forming the comma seperated string in the 'Rpayment' column.
For reference: https://stackoverflow.com/questions/58179593/how-to-combine-join-and-lambda-x-x-tolist-inside-an-groupby-agg-functio
Note: Don't forget to use 'reset_index' after using agg() here. 

3. Display "accept_payment_df"
"""

# YOUR CODE HERE


In [ ]:
##########################
### TEST YOUR SOLUTION ###
##########################

assert_frame_equal(accept_payment_df, sol.accept_payment_df)
print("test accept_payment_df correct")


In [ ]:
"""
Part 02

1. Using "restaurant_location_df", group by the 'alcohol' column and store in a dataframe called
"grouped_alcohol_df"

2. From this new dataframe get a list of all unique values in the 'alcohol' column, and store this in a variable
called 'alcohol_serve_type'

3. Print out the variable 'alcohol_serve_type'
"""

# YOUR CODE HERE


In [ ]:
##########################
### TEST YOUR SOLUTION ###
##########################

assert_equal(list(alcohol_serve_type), list(sol.alcohol_serve_type))
print("test alcohol_serve_type correct")

In [ ]:
"""
Part 03

1. Create a new dataframe called "alcohol_service_df". This dataframe is a copy of "restaurant_location_df" with
only the 'name' and 'alcohol' columns
Hint: Use the .copy() method when creating a new dataframe from those 2 columns

2. We want to see how many restaurants serve 'alcohol' and how many do not. To do this we need to create a new
column 'is_alcohol_served' in the "alcohol_service_df" dataframe.  Set the column's default value to 1.

3. Change the value of 'is_alcohol_served' to 0 if the value in the column 'alcohol' is "no_alcohol_served".
Hint, use this useful link: https://datatofish.com/if-condition-in-pandas-dataframe/

3. Find the number of places where alcohol is served and store as 'num_with_alcohol'

4. Find the number of places where alcohol is NOT served and store as 'num_without_alcohol'

5. Print both 'num_with_alcohol' and 'num_without_alcohol'
"""

# YOUR CODE HERE


In [ ]:
##########################
### TEST YOUR SOLUTION ###
##########################

assert_frame_equal(alcohol_service_df, sol.alcohol_service_df)
print("test alcohol_service_df correct")

assert_equal(num_with_alcohol, sol.num_with_alcohol)
print("test num_with_alcohol correct")
assert_equal(num_without_alcohol, sol.num_without_alcohol)
print("test num_without_alcohol correct")


In [ ]:
"""
Part 04

1. Create a pivot table called "pivot_rating" using the dataframe "rating_final_df".
The index should be 'placeID'.  The values should be 'food_rating', 'rating', and 'service_rating'.
Note: This will show the average 'food_rating', 'rating', and 'service_rating' for each place.

2. Display the pivot table
"""

# YOUR CODE HERE


In [ ]:
##########################
### TEST YOUR SOLUTION ###
##########################

assert_frame_equal(pivot_rating, sol.pivot_rating)
print("test pivot_rating correct")
